In [1]:
import pandas as pd
import torch
from transformers import MobileBertTokenizer, MobileBertModel
import string
from datasets import Dataset as Hug_Face_Dataset,load_from_disk
from torch.utils.data import DataLoader,Dataset
import torch.nn.functional as F
import torch.nn as nn      
from sklearn.metrics import precision_recall_fscore_support
from tqdm import tqdm

In [2]:
df = pd.read_parquet(r"/kaggle/input/parquet")

In [4]:
test_df = pd.read_parquet(r"/kaggle/input/testdata")

In [5]:
test_df

,tokens,ner_tags,lang
0,"[On, this, occasion, he, failed, to, gain, the...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 4, ...",en
1,"[On, both, these, occasions, he, was, backed, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 4, 4, 0, ...",en
2,"[He, also, appeared, as, himself, in, the, 199...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0]",en
3,"[The, Colorado, Rockies, were, created, as, an...","[0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 6, 0, ...",en
4,"[He, kept, busy, recording, demo, tapes, at, h...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",en
...,...,...,...
11592,"[com, ,, Amazon, .]","[0, 0, 3, 0]",en
11593,"[In, January, 2013, ,, the, European, Food, Sa...","[0, 0, 0, 0, 0, 3, 4, 4, 4, 0, 0, 0, 0, 0, 3, ...",en
11594,"[All, of, the, games, had, art, true, to, the,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, ...",en
11595,"[There, was, also, a, game, made, for, the, Ga...","[0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, ...",en


In [112]:
df.head()

,tokens,ner_tags,lang
0,"[This, division, also, contains, the, Ventana,...","[0, 0, 0, 0, 0, 5, 6, 0, 0, 0, 0, 5, 0, 0]",en
1,"["", So, here, is, the, balance, NBC, has, to, ...","[0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 7, 8, 0, 0, ...",en
2,"[It, is, a, protest, song, that, "", creates, a...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",en
3,"[This, differs, from, approaches, such, as, IP...","[0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, ...",en
4,"[Since, then, ,, only, Terry, Bradshaw, in, 14...","[0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 1, 2, 0, 0, 0, ...",en


In [9]:
mp2 = {'B-PER':1,'I-PER':2,'B-ORG':3,'I-ORG':4,'B-LOC':5,'I-LOC':6,'B-MISC':7,'I-MISC':8,'B-NRM':9,'B-REG':10,'B-RS':11,'I-LIT':12,'I-NRM':13,'I-REG':14,'I-RS':15,'B-ANIM':16,'I-ANIM':17,'B-BIO':18,'I-BIO':19,'B-CEL':20,'I-CEL':21,'B-DIS':22,'I-DIS':23,'B-EVE':24,
      'I-EVE':25,'B-FOOD':26,'I-FOOD':27,'B-INST':28,'I-INST':29,'B-MEDIA':30,'I-MEDIA':31,'B-MYTH':32,'I-MYTH':33,'B-PLANT':34,'I-PLANT':35,'B-TIME':36,'I-TIME':37,'B-VEHI':38,'I-VEHI':39,'B-LIT':40} 

In [7]:
def fun(tokens,tags):
    count = 0
    words = []
    entities = []
    index = 0
    for token in tokens:
        if(token[0] not in string.punctuation):
            temp = token.split()
            for t in temp:
                words.append(t)
            if(tags[index]!=0):
                entities.append((str(count+1),str(count+len(temp)),mp[tags[index]]))
            count+=len(temp)
        index+=1
    return words,entities

In [8]:
df2 = pd.DataFrame(columns=['input','output'])

In [9]:
test_df2 = pd.DataFrame(columns=['input','output'])

In [10]:
test_df2

,input,output


In [11]:
df2

,input,output


In [12]:
mp = {1:'B-PER',2:'I-PER',3:'B-ORG',4:'I-ORG',5:'B-LOC',6:'I-LOC',7:'B-MISC',8:'I-MISC'}

In [13]:
for i in range(0,len(df)):
    k,l = fun(df['tokens'][i],df['ner_tags'][i])
    df2.loc[i]=[k,l]

In [14]:
for i in range(0,len(test_df)):
    k,l = fun(test_df['tokens'][i],test_df['ner_tags'][i])
    test_df2.loc[i]=[k,l]

In [15]:
test_df2

,input,output
0,"[On, this, occasion, he, failed, to, gain, the...","[(12, 12, B-ORG), (13, 13, I-ORG), (14, 14, I-..."
1,"[On, both, these, occasions, he, was, backed, ...","[(10, 10, B-ORG), (11, 11, I-ORG), (12, 12, I-..."
2,"[He, also, appeared, as, himself, in, the, 199...","[(10, 10, B-MISC)]"
3,"[The, Colorado, Rockies, were, created, as, an...","[(2, 2, B-ORG), (3, 3, I-ORG), (13, 13, B-LOC)..."
4,"[He, kept, busy, recording, demo, tapes, at, h...","[(24, 24, B-LOC), (25, 25, I-LOC), (26, 26, I-..."
...,...,...
11592,"[com, Amazon]","[(2, 2, B-ORG)]"
11593,"[In, January, 2013, the, European, Food, Safet...","[(5, 5, B-ORG), (6, 6, I-ORG), (7, 7, I-ORG), ..."
11594,"[All, of, the, games, had, art, true, to, the,...","[(12, 12, B-ORG)]"
11595,"[There, was, also, a, game, made, for, the, Ga...","[(9, 9, B-MISC), (10, 10, I-MISC)]"


In [16]:
df2

,input,output
0,"[This, division, also, contains, the, Ventana,...","[(6, 6, B-LOC), (7, 7, I-LOC), (11, 11, B-LOC)]"
1,"[So, here, is, the, balance, NBC, has, to, con...","[(6, 6, B-ORG), (10, 10, B-MISC), (11, 11, I-M..."
2,"[It, is, a, protest, song, that, creates, a, c...","[(22, 22, B-LOC)]"
3,"[This, differs, from, approaches, such, as, IP...","[(9, 9, B-MISC)]"
4,"[Since, then, only, Terry, Bradshaw, in, 147, ...","[(4, 4, B-PER), (5, 5, I-PER), (9, 9, B-PER), ..."
...,...,...
92715,"[The, couple, had, a, son, David, and, a, daug...","[(6, 6, B-PER), (10, 10, B-PER)]"
92716,"[The, Home, Secretary, J., R., Clynes, was, pr...","[(4, 4, B-PER), (5, 5, I-PER), (6, 6, I-PER)]"
92717,"[At, the, time, of, her, birth, she, was, four...","[(17, 17, B-PER), (18, 18, I-PER)]"
92718,"[The, film, was, based, on, the, Broadway, pla...","[(7, 7, B-MISC)]"


In [17]:
dataset3 = Hug_Face_Dataset.from_pandas(df2)

In [18]:
test_dataset3 = Hug_Face_Dataset.from_pandas(test_df2)

In [19]:
dataset3.save_to_disk(r"C:\Users\srira\OneDrive\Desktop\kaggle")

Saving the dataset (0/1 shards):   0%|          | 0/92720 [00:00<?, ? examples/s]

In [20]:
test_dataset3.save_to_disk(r"C:\Users\srira\OneDrive\Desktop\test_kaggle")

Saving the dataset (0/1 shards):   0%|          | 0/11597 [00:00<?, ? examples/s]

In [92]:
dataset = load_from_disk(r"C:\Users\srira\OneDrive\Desktop\kaggle")

In [ ]:
dataset2 = load_from_disk(r"C:\Users\srira\OneDrive\Desktop\test_kaggle")

In [93]:
df_temp = pd.DataFrame(dataset)

In [ ]:
df_temp_2 = pd.DataFrame(dataset2)

In [ ]:
df_temp_2

In [94]:
df_temp

,input,output,__index_level_0__
0,"[This, division, also, contains, the, Ventana,...","[[6, 6, B-LOC], [7, 7, I-LOC], [11, 11, B-LOC]]",0
1,"[So, here, is, the, balance, NBC, has, to, con...","[[6, 6, B-ORG], [10, 10, B-MISC], [11, 11, I-M...",1
2,"[It, is, a, protest, song, that, creates, a, c...","[[22, 22, B-LOC]]",2
3,"[This, differs, from, approaches, such, as, IP...","[[9, 9, B-MISC]]",3
4,"[Since, then, only, Terry, Bradshaw, in, 147, ...","[[4, 4, B-PER], [5, 5, I-PER], [9, 9, B-PER], ...",4
...,...,...,...
92715,"[The, couple, had, a, son, David, and, a, daug...","[[6, 6, B-PER], [10, 10, B-PER]]",92715
92716,"[The, Home, Secretary, J., R., Clynes, was, pr...","[[4, 4, B-PER], [5, 5, I-PER], [6, 6, I-PER]]",92716
92717,"[At, the, time, of, her, birth, she, was, four...","[[17, 17, B-PER], [18, 18, I-PER]]",92717
92718,"[The, film, was, based, on, the, Broadway, pla...","[[7, 7, B-MISC]]",92718


In [95]:
class MyDataset(Dataset):
    def __init__(self, inputTokens,outputEntity,tokenizer,max_tokens=128,mapping={'B-PER':1}):
        self.source = inputTokens
        self.target = outputEntity
        self.tokenizer = tokenizer
        self.max_tokens = max_tokens
        self.mapping = mapping
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
    def __len__(self):
        return len(self.source)
    
    def __getitem__(self, idx):
        
        source_data = self.source[idx]
        target_data = self.target[idx]
        tokenized_input = self.tokenizer(source_data, return_tensors='pt', padding='max_length', truncation=True, max_length=self.max_tokens,is_split_into_words=True)
        label_ids = torch.zeros(tokenized_input['input_ids'].size(1), dtype=torch.long)
        
        for start, end, label in target_data:
            start = int(start)
            end = int(end)
            label_id = self.mapping[label]
            label_ids[start:end+1] = label_id
        
        return tokenized_input['input_ids'].squeeze(0).to(self.device), tokenized_input['attention_mask'].squeeze(0).to(self.device), label_ids.to(self.device)

In [129]:
class NERModel2(nn.Module):
    
    def __init__(self, max_tokens=128, hidden_size=64, num_entities=8,mobilebert_model=None,device='cpu',dropout_p=0.5):
        
        super(NERModel2, self).__init__()
        
        self.max_tokens = max_tokens
        self.num_entities = num_entities
        # MobileBERT model (base model)
        self.mobilebert = mobilebert_model.to(device=device)
        
        self.hidden_size = hidden_size
        
        self.device = device
        
        # First GRU Layer
        
        self.gru1 = nn.GRU(input_size=self.mobilebert.config.hidden_size, hidden_size=hidden_size, batch_first=True)
        
        # Second GRU Layer
        
        self.gru2 = nn.GRU(input_size=hidden_size, hidden_size=hidden_size, batch_first=True)
        
        # Fully connected layers
        self.fc_layer = nn.Linear(hidden_size, num_entities)
        
        self.dropout = nn.Dropout(dropout_p)
    
            
    def forward(self, input_ids, attention_mask):
        
        batch_size = input_ids.size(0)
        
        
        outputs = self.mobilebert(input_ids=input_ids, attention_mask=attention_mask)
        
        sequence_output = outputs.last_hidden_state  
        
        temp = torch.zeros(1,batch_size,self.hidden_size,device = input_ids.device)
        
        
        gru1_outputs = torch.zeros(batch_size, self.max_tokens, self.gru1.hidden_size, device=input_ids.device)
        gru2_outputs = torch.zeros(batch_size, self.max_tokens, self.gru2.hidden_size, device=input_ids.device)
        
        for i in range(self.max_tokens):

            gru1_output, temp = self.gru1(sequence_output[:, i].unsqueeze(1),temp)
            gru1_outputs[:,i,:] = gru1_output.squeeze(1)
            
        temp = torch.zeros(1,batch_size,self.hidden_size,device = input_ids.device)
        
        for i in range(self.max_tokens-1,-1,-1):
            temp2, temp =  self.gru1(sequence_output[:,i].unsqueeze(1),temp)
            temp2 = temp2.squeeze(1)
            gru1_outputs[:,i,:] = (gru1_outputs[:,i,:] + temp2) / 2
            
        temp = torch.zeros(1,batch_size,self.gru2.input_size,device = input_ids.device)
        
        for i in range(self.max_tokens):
            gru2_output, temp = self.gru2(gru1_outputs[:, i].unsqueeze(1),temp)
            gru2_outputs[:,i,:] = gru2_output.squeeze(1)
            
        temp = torch.zeros(1,batch_size,self.gru2.input_size,device = input_ids.device)
        
        for i in range(self.max_tokens-1,-1,-1):
            temp2, temp =  self.gru2(gru1_outputs[:,i].unsqueeze(1),temp)
            temp2 = temp2.squeeze(1)
            gru2_outputs[:,i,:] = (gru2_outputs[:,i,:] + temp2) / 2                  
            
        overallOutput = torch.zeros(batch_size, self.max_tokens,self.num_entities, device=input_ids.device)
        
        for i in range(self.max_tokens):
            output = self.dropout(gru2_outputs[:, i, :]) # dropout on gru outputs
            overallOutput[:, i, :] = self.fc_layer(output)
        
        return overallOutput

In [97]:
def dataProcessing(dataset,tokenizer,max_tokens,batch_size,mapping):
    inputData = dataset['input'] 
    outputData = dataset['output']
    myDataset = MyDataset(inputData,outputData,tokenizer,max_tokens,mapping)
    dataLoader = DataLoader(myDataset,batch_size,shuffle = True)
    return dataLoader    

In [119]:
def testingAccuracyCalculator(model,tokenizer,max_tokens,batch_size,mapping):
    
    dataset = load_from_disk(r"C:\Users\srira\OneDrive\Desktop\test_kaggle")
    
    dataLoader = dataProcessing(dataset,tokenizer,max_tokens,batch_size,mapping)
    
    criterion = nn.CrossEntropyLoss()
    numberOfInputs = len(dataset['input'])*128
    total_loss = 0.0
    total_steps = 0.0
    correct_predictions = 0.0        
    
    all_true_labels = []
    all_pred_labels = []
    
    model.eval() #model is now in evaluation mode. 
    
    
    for idx,(input_ids, attention_mask, labels) in enumerate(tqdm(dataLoader)):

        output = model(input_ids, attention_mask)

        # Flatten the logits and labels for loss calculation

        output_flat = output.view(-1, output.size(-1))
        labels_flat = labels.view(-1)

        probabilities = F.softmax(output_flat, dim=1)

        predicted_labels = torch.argmax(probabilities, dim=1)

        correct_predictions += (predicted_labels == labels_flat).sum().item()

        all_true_labels.extend(labels_flat.cpu().numpy())
        all_pred_labels.extend(predicted_labels.cpu().numpy())

        loss = criterion(output_flat, labels_flat)
        
        total_loss += loss.item() 

        if(idx%100==0):
            print(f'loss {loss.item()}')

        total_steps += 1
        
    avg_loss = total_loss / total_steps
    accuracy_percentage = (correct_predictions / numberOfInputs)*100
    precision, recall, f1, _ = precision_recall_fscore_support(all_true_labels, all_pred_labels, average='weighted',zero_division=0) #I have used weighted average of three scores.
    print(f'average_loss {avg_loss}, accuracyPercentage, {accuracy_percentage:.2f}')  
    print(f'precision{precision:.2f},recall{recall:.2f},f1_score{f1:.2f}')
    with open(r"C:\Users\srira\OneDrive\Desktop\kaggle\checkpoints.log", 'a') as log_file:
        log_file.write(f'Testing mode ->> avg_loss {avg_loss}, accuracy_percentage {accuracy_percentage:.2f},precision {precision:.2f},recall {recall:.2f}, f1 {f1:.2f}\n')
    model.train()  # model now is in training mode.

In [154]:
def train(max_tokens,hidden_size,num_entities,learning_rate,batch_size,mapping,epochs,dropout):
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    tokenizer = MobileBertTokenizer.from_pretrained('google/mobilebert-uncased')
    mobilebert_model = MobileBertModel.from_pretrained('google/mobilebert-uncased')
    mobilebert_model.config.embedding_size = 256    
    dataset = load_from_disk(r"C:\Users\srira\OneDrive\Desktop\kaggle")
    
    dataloader = dataProcessing(dataset,tokenizer,max_tokens,batch_size,mapping)
    
    model = NERModel2(max_tokens,hidden_size,num_entities,mobilebert_model,device,dropout)

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    criterion = nn.CrossEntropyLoss()
    
    model.to(device)
    
    model.train()
    
    numberOfInputs = len(dataset['input'])*128
    
    with open(r"C:\Users\srira\OneDrive\Desktop\kaggle\checkpoints.log", 'w') as log_file:
        log_file.write("Starting training log\n")
    
    for epoch in range(epochs): 

        total_loss = 0.0
        total_steps = 0.0
        
        correct_predictions = 0.0
        print(f'epoch{epoch}')
        
        all_true_labels = []
        all_pred_labels = []
        
        for idx,(input_ids, attention_mask, labels) in enumerate(tqdm(dataloader)):
            
            optimizer.zero_grad()
        
            output = model(input_ids, attention_mask)

            # Flatten the logits and labels for loss calculation
            
            output_flat = output.view(-1, output.size(-1))
            labels_flat = labels.view(-1)

            probabilities = F.softmax(output_flat, dim=1)

            predicted_labels = torch.argmax(probabilities, dim=1)
            
            correct_predictions += (predicted_labels == labels_flat).sum().item()
            
            all_true_labels.extend(labels_flat.cpu().numpy())
            all_pred_labels.extend(predicted_labels.cpu().numpy())

            # Calculate loss
            loss = criterion(output_flat, labels_flat)

            # Backward pass
            loss.backward()
            
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0) # gradient clipping
            
            optimizer.step()

            # Accumulate loss
            total_loss += loss.item() 
            
            if(idx%200==0):
                print(f'loss {loss.item()}')
            
            total_steps += 1

        # Print average loss
        avg_loss = total_loss / total_steps
        accuracy_percentage = (correct_predictions / numberOfInputs)*100
        precision, recall, f1, _ = precision_recall_fscore_support(all_true_labels, all_pred_labels, average='weighted',zero_division=0) #I have used weighted average of three scores.
        print(f'epoch {epoch}, average_loss {avg_loss}, accuracyPercentage, {accuracy_percentage:.2f}')  
        print(f'precision{precision:.2f},recall{recall:.2f},f1_score{f1:.2f}')
        with open(r"C:\Users\srira\OneDrive\Desktop\kaggle\checkpoints.log", 'a') as log_file:
            log_file.write(f'Training mode ->> epoch {epoch},avg_loss {avg_loss}, accuracy_percentage {accuracy_percentage:.2f},precision {precision:.2f},recall {recall:.2f}, f1 {f1:.2f}\n')
        checkpoint_path = os.path.join(r"C:\Users\srira\OneDrive\Desktop\kaggle", f'epoch_{epoch}_checkpoint.pth')
        torch.save(model.state_dict(), checkpoint_path)
        testingAccuracyCalculator(model,tokenizer,max_tokens,batch_size,mapping)

In [151]:
import os

In [153]:
epoch = 1


In [33]:
mapping = {'B-PER':1,'I-PER':2,'B-ORG':3,'I-ORG':4,'B-LOC':5,'I-LOC':6,'B-MISC':7,'I-MISC':8,'B-NRM':9,'B-REG':10,'B-RS':11,'I-LIT':12,'I-NRM':13,'I-REG':14,'I-RS':15,'B-ANIM':16,'I-ANIM':17,'B-BIO':18,'I-BIO':19,'B-CEL':20,'I-CEL':21,'B-DIS':22,'I-DIS':23,'B-EVE':24,
      'I-EVE':25,'B-FOOD':26,'I-FOOD':27,'B-INST':28,'I-INST':29,'B-MEDIA':30,'I-MEDIA':31,'B-MYTH':32,'I-MYTH':33,'B-PLANT':34,'I-PLANT':35,'B-TIME':36,'I-TIME':37,'B-VEHI':38,'I-VEHI':39,'B-LIT':40}

In [ ]:
print(64*)

In [ ]:
mapping = {'B-PER':1,'I-PER':2,'B-ORG':3,'I-ORG':4,'B-LOC':5,'I-LOC':6,'B-MISC':7,'I-MISC':8,'B-NRM':9,'B-REG':10,'B-RS':11,'I-LIT':12,'I-NRM':13,'I-REG':14,'I-RS':15,'B-ANIM':16,'I-ANIM':17,'B-BIO':18,'I-BIO':19,'B-CEL':20,'I-CEL':21,'B-DIS':22,'I-DIS':23,'B-EVE':24,
      'I-EVE':25,'B-FOOD':26,'I-FOOD':27,'B-INST':28,'I-INST':29,'B-MEDIA':30,'I-MEDIA':31,'B-MYTH':32,'I-MYTH':33,'B-PLANT':34,'I-PLANT':35,'B-TIME':36,'I-TIME':37,'B-VEHI':38,'I-VEHI':39,'B-LIT':40} 
train(max_tokens=128,hidden_size=64,num_entities=41,learning_rate=1e-4,batch_size=64,mapping=mapping,epochs=2,dropout=0.8)

epoch0


  0%|          | 1/1449 [00:01<26:32,  1.10s/it]

loss 3.9240734577178955


  0%|          | 6/1449 [00:06<24:33,  1.02s/it]